# Treinando o modelo na Versão 04 V2


In [1]:
import gymnasium as gym
print(f"Using gymnasium version {gym.__version__}")


Using gymnasium version 0.29.1


In [2]:
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback, CheckpointCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
import numpy as np
import os

print(f"Using stable-baselines3 version {stable_baselines3.__version__}")

Using stable-baselines3 version 2.1.0


In [3]:
from Enviroment.Settings import *
from Enviroment.Manager import Enviroment

In [4]:
import torch as th

print(f"Using torch version {th.__version__}")

Using torch version 2.0.1+cpu


In [5]:
class TensorboardCallback(BaseCallback):
    """
    Custom callback for plotting additional values in tensorboard.
    """

    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.pb_history = []

    def _on_step(self) -> bool:

        # Acessa o ambiente
        env = self.training_env.envs[0].env

        info = env.get_custom_info()

        if info["total_requests"] > 39998:
            self.logger.record("custom_on_episode/total_requests", info["total_requests"])
            self.logger.record("custom_on_episode/blocking_probability", info["blocking_probability"])
            self.pb_history.append(info["blocking_probability"])
            self.logger.record("custom_on_episode/blocking_probability_total_mean", np.mean(self.pb_history))

            # Salva a média da probabilidade de bloqueio dos últimos 10 episódios
            self.logger.record("custom_on_episode/blocking_probability_mean_10", np.mean(self.pb_history[-10:]))


            self.logger.record("custom_on_episode/reward_episode", info["reward_episode"])
            self.logger.record("custom_on_episode/total_number_of_blocks", info["total_number_of_blocks"])
            self.logger.record("custom_on_episode/calls_SAR", env.SAR_calls)

        # Verifica se o episódio terminou	
        return True

In [6]:
# Configura como deve ser o ambiente
enviroment_type = {
    "Observation": "ODD-one-hot",
    "Action": "RSA-SAR",
    "Reward": "RL-defaut",
    "StopCond": "40kReqs",
    "StartCond": "Empty"
}

# Cria o ambiente de simulação
env = Enviroment(
    network_load=300,
    k_routes=K_ROUTES,
    number_of_slots=NUMBER_OF_SLOTS,
    enviroment_type=enviroment_type,
    data_folder="Train_PPO_40kReqs_300_v19",
)

env_eval = Enviroment(
    network_load=300,
    k_routes=K_ROUTES,
    number_of_slots=NUMBER_OF_SLOTS,
    enviroment_type=enviroment_type,
    data_folder="Train_PPO_40kReqs_300_v19_eval",
)

# Cria o log path
LOG_PATH = env.folder_name

print(f"Using the enviroment {enviroment_type}")

print(f"Using the log path {LOG_PATH}")

# Criando o diretório para o monitoramento
monitor_dir = LOG_PATH + "\\monitor_logs"
os.makedirs(monitor_dir, exist_ok=True)

# Criando o monitor para monitorar o ambiente
env = Monitor(env, monitor_dir)

# Cria o dicionário com as configurações da política da rede. 
policy_kwargs = dict(
    activation_fn=th.nn.Tanh,
    net_arch=dict(pi=[256, 128], vf=[128, 64]))

# Create the agent
model = PPO(
    "MlpPolicy", 
    env,
    batch_size =2000,
    n_steps=80000,
    learning_rate=0.0001,
    policy_kwargs=policy_kwargs,
    verbose=0, 
    gamma=0.9999,
    tensorboard_log=LOG_PATH + '\\tensorboard\\', 
    seed=42
)

# Callback de avaliação
eval_callback = EvalCallback(
    Monitor(env_eval, monitor_dir + "\\eval"),  
    eval_freq=80_000, 
    deterministic=True, 
    best_model_save_path=LOG_PATH + '\\training\\best_model',
    log_path=LOG_PATH + '\\training\\logs',
    render=False, 
    verbose=0
)

model.learn(
    total_timesteps=12_000_000, 
    callback=[TensorboardCallback(), eval_callback],
    progress_bar=True, 
    tb_log_name="Model_v13_v1"
)

# Salva o modelo treinado
model.save(LOG_PATH + '\\training\\final_model')


d:\PythonEnvironment\PhDVenv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

Using the enviroment {'Observation': 'ODD-one-hot', 'Action': 'RSA-SAR', 'Reward': 'RL-defaut', 'StopCond': '40kReqs', 'StartCond': 'Empty'}
Using the log path D:\98_phD_Files\Projeto 006 - Artigo rede regular iTwo\logs\Train_PPO_40kReqs_300_v19_001
